In [1]:
import math
import matplotlib.pyplot as plt

In [2]:
%%capture
%run "6 - Probability.ipynb"

# Statistical Hypothesis Testing
Classical testing involves a _null hypothesis_ $H_0$ that represents the default and an alternative hypothesis $H_1$ to test. Statistics helps us to determine whether $H_0$ should be considered false or not.

## Example: Flipping A Coin
Null hypothesis = the coin is fair = $p = 0.5$
Alternative hypothesis = coins is not fair = $p \not = 0.5$

To test, $n$ samples will be collected. Each toss is a Bernoulli(n, p) trial.

In [5]:
def normal_approximation_to_binomial(n, p):
    """return mu and sigma corresponding to Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [3]:
normal_probability_below = normal_cdf

def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

def normal_probabilty_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

Now we flip the coin 1,000 times and see if our null hypothesis is true. If so, $X$ will be approximately normally distributed with a mean of 500 and a standard deviation of 15.8:

In [10]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
mu_0, sigma_0

(500.0, 15.811388300841896)

A decision must be made with respect to _significance_, i.e., how willing are we to accept "false positives" (_type 1 errors_) by rejecting $H_0$ even though it is true? This is often set to 5% or 1%. We will use 5%:

In [11]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

If $H_0$ is true, $p$ should equal 0.5. The interval above denotes the values outside of which there is a 5% chance that this is false. Now we can determine the _power_ of a test, i.e., how willing are we to accept "false positives" (_type 2 errors_) by failing to reject $H_0$ even though it is false.

In [12]:
# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# a type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability # 0.887

Run a 5% significance test to find the cutoff below which 95% of the probability lies:

In [13]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)

# is 526 (< 531, since we need more probability in the upper tail)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)

power = 1 - type_2_probability # 0.936

In [15]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)

two_sided_p_value(529.5, mu_0, sigma_0) # 0.062

0.06207721579598857

In [17]:
extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0 # count # of heads
                    for _ in range(1000)) # in 1000 flips
    if num_heads >= 530 or num_heads <= 470: # and count how often
        extreme_value_count += 1 # the # is 'extreme'

print(extreme_value_count / 100000) # 0.062

0.06233


In [18]:
two_sided_p_value(531.5, mu_0, sigma_0) # 0.0463

0.046345287837786575

Make sure your data is roughly normally distributed before using normal_probability_above to compute
p-values. The annals of bad data science are filled with examples of people opining that the chance of some
observed event occurring at random is one in a million, when what they really mean is “the chance,
assuming the data is distributed normally,” which is pretty meaningless if the data isn’t.

There are various statistical tests for normality, but even plotting the data is a good start.

# Confidence Intervals
We can construct a _confidence interval_ around an observed value of a parameter. We can do this for our assumption of an unfair coin (biased towards heads in that we observed 525 of 1,000 flips giving heads):

In [19]:
math.sqrt(p * (1 - p) / 1000)

NameError: name 'p' is not defined

Not knowing $p$ we use our estimate:

In [21]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
sigma

0.015791611697353755

Assuming a normal distribution, we conclude that we are 95% confident that the interval below includes the true $p$:

In [22]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

0.5 falls within our interval so we do not conclude that the coin is unfair.

What if we observe 540 heads though?

In [23]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000) # 0.0158
normal_two_sided_bounds(0.95, mu, sigma) # [0.5091, 0.5709]

(0.5091095927295919, 0.5708904072704082)

In this scenario, 0.5 falls outside of our interval so the "fair coin" hypothesis is not confirmed.